<a href="https://colab.research.google.com/github/gabicavalcante/nfce-manager-/blob/master/nfe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
nfs = [
    {
        "date": "2020-06-06",
        "local": "supermercado X",
        "url": "http://nfce.set.rn.gov.br/...."
    },
]

In [195]:
!pip install rows

## Import Libraries

In [0]:
from io import BytesIO

import requests
import six

import rows

from collections import OrderedDict

## Extract data

In [0]:
def get_items_csv(url):
  # Get the HTML
  response = requests.get(url)
  html = response.content

  # Import data, preserving cell's HTML
  packages = rows.import_from_html(BytesIO(html), index=4)
  return rows.export_to_csv(packages)

In [0]:
def get_total(url):
  total_table = rows.import_from_html(BytesIO(html), index=5)

  {([*element._asdict().values()][0]): ([*element._asdict().values()][1]) for element in total_table}

In [0]:
import pandas as pd
import io
import datetime 

df = pd.DataFrame({})
for nf in nfs:
  csv = io.StringIO(get_items_csv(nf["url"]).decode('utf-8'))
  df_nf = pd.read_csv(csv, usecols=["item", "descricao", "qtde", "unid", "vl_unid", "vl_total"])
  df_nf = df_nf.assign(
      local=nf["local"],
      date=pd.to_datetime(nf["date"])
  )
  df = pd.concat([df, df_nf])

df = df.sort_values(by='date').reset_index(drop=True)

In [0]:
wd = ("seg", "ter", "qua", "qui", "sex", "sab", "dom")

df["weekday"] = df.apply(lambda row: wd[row.date.weekday()], axis=1)

In [0]:
def get_item(item):
  my_mask = df["descricao"].str.contains(item, case=False)
  return df.query("@my_mask")

In [267]:
get_item("tomate")

,item,descricao,qtde,unid,vl_unid,vl_total,local,date,weekday
25,18.0,TOMATE RASTEIRO KG,"0,235",kg,"7,99","1,88",carrefour,2020-03-01,dom
61,33.0,TOMATE RASTEIRO KG,"0,385",kg,"5,99","2,31",carrefour,2020-03-11,qua
67,38.0,TOMATE RASTEIRO KG,"0,305",kg,"5,99","1,83",carrefour,2020-03-11,qua
88,8.0,TOMATE LONGA VIDA KG,"0,18",KG,"6,99","1,26",redemais,2020-03-19,qui
160,4.0,TOMATE LONGA VIDA KG,"0,635",KG,"6,99","4,44",redemais,2020-03-27,sex
194,7.0,MOLHO TOMATE TAMBAU 340G MANJERICAO,"1,0",UN,"1,45","1,45",redemais,2020-04-03,sex
196,6.0,MOLHO TOMATE TAMBAU 340G MANJERICAO,"1,0",UN,"1,45","1,45",redemais,2020-04-03,sex
210,27.0,TOMATE LONGA VIDA KG,"0,285",KG,"5,99","1,71",redemais,2020-04-03,sex
224,6.0,TOMATE LONGA VIDA KG,"0,235",KG,"6,99","1,64",redemais,2020-04-10,sex
296,33.0,TOMATE RASTEIRO KG,"0,345",kg,"7,99","2,76",carrefour,2020-05-31,dom


In [268]:
get_item("cerveja|brahma|beats")

,item,descricao,qtde,unid,vl_unid,vl_total,local,date,weekday
24,1.0,CERVEJA BRAHMA LT 35,"2,0",un,"1,79","3,58",carrefour,2020-03-01,dom
103,23.0,CERVEJA BRAHMA 300ML CHOPP RETORNAVEL,"4,0",UN,"1,99","7,96",redemais,2020-03-19,qui
121,9.0,CERVEJA PETRA 350ML PURO MALTE LT,"4,0",UN,"2,69","10,76",redemais,2020-03-19,qui
124,12.0,CERVEJA BRAHMA 473ML LATAO,"2,0",UN,"2,99","5,98",redemais,2020-03-19,qui
148,17.0,CERVEJA BRAHMA 473ML LATAO,"1,0",UN,"2,99","2,99",redemais,2020-03-27,sex
149,16.0,CERVEJA PETRA 350ML PURO MALTE LT,"4,0",UN,"2,69","10,76",redemais,2020-03-27,sex
193,10.0,CERVEJA SKOL 269ML BEATS SENSES LT,"1,0",UN,"4,49","4,49",redemais,2020-04-03,sex
200,9.0,CERVEJA SKOL 269ML BEATS SENSES LT,"1,0",UN,"4,49","4,49",redemais,2020-04-03,sex
220,11.0,CERVEJA SKOL 269ML BEATS SENSES LT,"1,0",UN,"4,49","4,49",redemais,2020-04-10,sex
226,8.0,CERVEJA BRAHMA 300ML CHOPP RETORNAVEL,"2,0",UN,"2,99","5,98",redemais,2020-04-10,sex
